<a href="https://colab.research.google.com/github/lcamposgarrido/PGPortfolio/blob/master/viva_investing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

     |████████████████████████████████| 5.5MB 11.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=2e4b52aaa8419d047eb9502c04e8d67e014df0cdcd6353dec8bbc700faa1b920
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
tickers = ["VUSA.AS", "EUE.MI", "EMAE.MI", "XDJP.MI", "BTC-EUR"]
for ticker in tickers:
  fund = yf.Ticker(ticker)
  print_important_info(fund)

Name: VANGUARD S&P500
Symbol: VUSA.AS
Type: ETF
Currency: EUR
Market Price: 54.092
Name: ISHARES EURO STOXX 50 UCITS ETF
Symbol: EUE.MI
Type: ETF
Currency: EUR
Market Price: 30.9
Name: SPDR MSCI EM ASIA UCITS ETF
Symbol: EMAE.MI
Type: ETF
Currency: EUR
Market Price: 67.37
Name: XTRACKERS NIKKEI 225 UCITS ETF
Symbol: XDJP.MI
Type: ETF
Currency: EUR
Market Price: 20.075
Name: Bitcoin EUR
Symbol: BTC-EUR
Type: CRYPTOCURRENCY
Currency: EUR
Market Price: 11603.689


In [ ]:
def print_important_info(fund):
  print("Name: " + fund.get_info()["shortName"])
  print("Symbol: " + fund.get_info()["symbol"])
  print("Type: " + fund.get_info()["quoteType"])
  print("Currency: " + fund.get_info()["currency"])
  #print(f"Total Assets: {str(fund.get_info()['totalAssets'] / 10e6)} M")
  #print("Market Volume: " + str(fund.get_info()["regularMarketVolume"]))
  print(f"Market Price: {str(fund.get_info()['regularMarketPrice'])}")
  #print(f"3Y returns: {str(fund.get_info()['threeYearAverageReturn'] * 100)}%")
  #print(f"5Y returns: {str(fund.get_info()['fiveYearAverageReturn'] * 100)}%")


In [ ]:
def viva_advisory(invested_amount=1000, style="adventurous"):

  if style == "adventurous":
    categories = ["US Stocks", "Europe Stocks", "Asia Emerging Markets Stocks", "Japan Stocks", "Alternative", "Cash"]
    weights_limit = [0.338720316622691, 0.310026385224274, 0.26929419525066, 0.0819591029023747, 0.1, 0]
    names = [
      "Vanguard S&P 500 UCITS ETF",
      "iShares Core MSCI Europe UCITS ETF",
      "SPDR® MSCI EM Asia UCITS ETF",
      "Xtrackers Nikkei 225 UCITS ETF",
      "Bitcoin",
      "Euro",
    ]
    symbols = ["VUSA.AS", "IQQY.DE", "EMAE.MI", "XDJP.MI", "BTC-EUR", "EUR"]
    codes = ["IE00B3XXRP09", "IE00B1YZSC51", "IE00B466KX20", "LU0839027447", "BTC", "EUR"]
  elif style == "cautious":
    categories = ["US Stocks", "Europe Stocks", "Asia Emerging Markets Stocks", "Japan Stocks", "Europe Corporate Bonds", "Europe Government Bonds", "Alternative", "Cash"]
    weights_limit = [0.338720316622691 * 0.6, 0.310026385224274 * 0.6, 0.26929419525066 * 0.6, 0.0819591029023747 * 0.6, 0.2, 0.2, 0.1, 0]
    names = [
      "Vanguard S&P 500 UCITS ETF",
      "iShares Core MSCI Europe UCITS ETF",
      "SPDR® MSCI EM Asia UCITS ETF",
      "Xtrackers Nikkei 225 UCITS ETF",
      "iShares Core € Corp Bond UCITS ETF EUR",
      "iShares € Govt Bond 3-5yr UCITS ETF EUR",
      "Physical Gold",
      "Euro",
    ]
    symbols = ["VUSA.AS", "IQQY.DE", "EMAE.MI", "XDJP.MI", "IEAC.MI", "IBGX.MI", "GC=F", "EUR"]
    codes = ["IE00B3XXRP09", "IE00B1YZSC51", "IE00B466KX20", "LU0839027447", "IE00B3F81R35", "IE00B1FZS681", "GC", "EUR"]
  prices = list()
  for symbol in symbols:
    asset = yf.Ticker(symbol)
    price = asset.info["regularMarketPrice"]
    prices.append(price)

  advisory_table = pd.DataFrame([names, codes, categories, weights_limit, symbols, prices]).transpose()
  advisory_table.columns = ["Name", "Code", "Category", "Weight limit", "Symbol", "Price"]

  # Set cash price to 1
  cash_index = advisory_table[advisory_table["Category"]=="Cash"].index[0]
  advisory_table.at[cash_index, "Price"] = 1

  shares = invested_amount / advisory_table["Price"] * advisory_table["Weight limit"]
  advisory_table["Shares"] = np.floor(shares)
  amount = advisory_table["Price"] * advisory_table["Shares"]
  advisory_table["Amount"] = amount
  weight = advisory_table["Amount"] / invested_amount
  advisory_table["Weight"] = weight

  # Set alternative weight to minimum between rest and 10%
  rest_weight = 1 - advisory_table["Weight"].sum()
  alternative_weight = min(rest_weight, 0.1)
  alternative_index = advisory_table[advisory_table["Category"]=="Alternative"].index[0]
  advisory_table.at[alternative_index, "Weight"] = alternative_weight
  advisory_table.at[alternative_index, "Amount"] = alternative_weight * invested_amount
  advisory_table.at[alternative_index, "Shares"] = advisory_table.at[alternative_index, "Weight"] * invested_amount / advisory_table.at[alternative_index, "Price"]

  # Set rest to cash
  rest_weight = 1 - advisory_table["Weight"].sum()
  advisory_table.at[cash_index, "Weight"] = rest_weight
  advisory_table.at[cash_index, "Amount"] = rest_weight * invested_amount
  advisory_table.at[cash_index, "Shares"] = advisory_table.at[cash_index, "Weight"] * invested_amount / advisory_table.at[cash_index, "Price"]

  return advisory_table

def format_table(advisory_table):
  advisory_table = advisory_table[["Name", "Code", "Category", "Shares", "Amount"]]
  cash_index = advisory_table[advisory_table["Category"]=="Cash"].index[0]
  advisory_table.at[cash_index, "Shares"] = ""
  alternative_index = advisory_table[advisory_table["Category"]=="Alternative"].index[0]
  advisory_table.at[alternative_index, "Shares"] = ""
  #advisory_table["Amount"] = advisory_table["Amount"].astype(float).round(2)
  advisory_table["Amount"] = advisory_table["Amount"].apply(lambda x: "{0:.2f} €".format(x))

  return advisory_table

for style in ["cautious", "adventurous"]:
  for invested_amount in [250, 500, 1000, 2000]:
    advisory_table = viva_advisory(invested_amount, style)
    advisory_chart = advisory_table[["Category", "Weight"]]
    advisory_chart.loc[-1] = ["string", "number"]
    advisory_chart.sort_index(inplace=True) 
    # Save table
    advisory_table = format_table(advisory_table)
    advisory_table.to_csv(f"advisory_table_{style}_{invested_amount}.csv", index=False)
    # Save chart
    advisory_chart.to_csv(f"advisory_chart_{style}_{invested_amount}.csv", index=False)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/lo